# **OJK ChatBot - LangChain**

## **Setup**

In [4]:
from dotenv import load_dotenv
load_dotenv()

True

## **Config**

In [5]:
from utils.config import get_config
from utils.questions import get_question
from utils.model_config import ModelName, get_model

config = get_config()

In [6]:
STORE = False
DELETE = False
K_STORE = 10
TOP_K = 10
TOP_N = 6
model_name = ModelName.AZURE_OPENAI
query_str = get_question("k")
# query_str = "Apa judul peraturan nomor 37 /SEOJK.03/2016?"

USER_ID = "xmriz"
CONVERSATION_ID = "xmriz-1"

## **Define Model**

In [7]:
llm_model, embed_model = get_model(model_name=model_name, config=config)

## **Indexing**

### **Load**

In [8]:
from utils.documents_text_extract import extract_all_documents_in_directory

documents_dir = './data/documents'
metadata_path = './data/metadata/files_metadata.csv'

if STORE:
    documents = extract_all_documents_in_directory(documents_dir, metadata_path, treshold=0.98)


### **Split**

In [9]:
from utils.document_split import document_splitter

if STORE:
    all_splits = document_splitter(docs=documents)

### **Storing**

In [10]:
from utils.vector_store import PineconeIndexManager

pinecone = PineconeIndexManager(index_name='ojk', embed_model=embed_model, config=config)

if STORE:
    pinecone.store_vector_index(docs=all_splits, delete=DELETE)
    vector_store = pinecone.load_vector_index()
else: 
    vector_store = pinecone.load_vector_index()

## **Retrieval and Generation**

### **Retrieve**

In [11]:
from langchain_cohere import CohereRerank
from langchain.retrievers import ContextualCompressionRetriever

# retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": TOP_K})
compressor = CohereRerank(cohere_api_key=config['cohere_api_key'], top_n=TOP_N)
retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=vector_store.as_retriever(search_type="similarity", search_kwargs={"k": TOP_K})
)

In [12]:
retrieved_docs = retriever.invoke(input=query_str)
retrieved_docs

[Document(metadata={'effective_date': '26 Februari 2008', 'file_url': 'https://www.ojk.go.id/id/regulasi/Documents/Pages/PMK-Nomor-36PMK.010-Tahun-2008-tentang-Besar-Santunan-dan-Sumbangan-Wajib-Dana-Kecelakaan-Lalu-Lintas-Jalan/menas13_1389258036.pdf', 'page_number': 4.0, 'regulation_number': '36/PMK.010/2008', 'regulation_type': 'Klasifikasi Bapepam', 'sector': 'IKNB', 'subsector': 'Asuransi', 'title': 'Peraturan Menteri Keuangan Nomor 36/PMK.010/2008 tentang Besar Santunan dan Sumbangan Wajib Dana Kecelakaan Lalu Lintas Jalan', 'relevance_score': 0.9850429}, page_content='2) Dalam hal pembayaran SWDKLLJ dilakukan setelah melewati batas\n seharusnya dibayar dengan ketentuan denda yang dikenakan paling\nbesar Rp100.000,00 (seratus ribu rupiah).\n(3) Dalam hal ketentuan mengenai batas waktu sebagaimana dimaksud\ngeografis daerah setempat, Direksi perusahaan yang ditunjuk untuk\nmenyelenggarakan Dana Kecelakaan Lalu Lintas Jalan diberi\nbesarnya denda SWDKLLJ, dengan ketentuan batas wak

### **Generate**

In [13]:
PROMPT = """The context information is below.
Context: 
{context}

Based on the context and the metadata information provided, \
answer the query related to banking compliance in Indonesia.
Use the context and metadata information only, without relying on external sources. 
ALWAYS ANSWER IN THE USER'S LANGUAGE.

Please provide your answer in the following format, \
including the regulation number and file URL if available:

[Your answer here] \n\n
Source: [metadata['regulation_number']](metadata['file_url'])

If you cannot find the regulation number, just provide the answer without the source. 
If the file_url ends with '.pdf', you can add the metadata['page_number'] in the URL like this: 

[Your answer here] \n\n
Source: [metadata['regulation_number']](metadata['file_url#page=metadata['page_number']')

DO NOT PROVIDE AMBIGUOUS ANSWERS.
DO NOT ANSWER THE QUESTION THAT IS NOT RELATED TO THE CONTEXT.
"""

In [14]:
from utils.chat_history import ChatStore

chat_store = ChatStore(k=K_STORE)

In [15]:
from utils.rag_chain_with_chat_history import create_chain_with_chat_history

chain = create_chain_with_chat_history(
    prompt_str=PROMPT,
    retriever=retriever,
    get_session_history=chat_store.get_session_history,
    llm_model=llm_model,
)

**With Context, and History**

In [34]:
# from utils.rag_chain_with_chat_history import get_response

# response = get_response(
#     chain=chain,
#     question=query_str,
#     user_id="xmriz",
#     conversation_id="conv123",
# )

# response

In [35]:
# print(chat_store.get_session_history("xmriz", "conv123"))

**Streaming**

In [69]:
from utils.rag_chain_with_chat_history import print_answer_stream

print_answer_stream(
    question="Siapa itu Lionel Messi?",
    chain=chain,
    user_id=USER_ID,
    conversation_id=CONVERSATION_ID,
)

Lionel Messi adalah seorang pemain sepak bola profesional asal Argentina. Ia dikenal sebagai salah satu pemain terbaik dalam sejarah sepak bola. Messi memulai karir profesionalnya di klub Barcelona, di mana ia menghabiskan sebagian besar kariernya. Selama bermain untuk Barcelona, Messi telah memenangkan banyak gelar dan penghargaan, termasuk enam Ballon d'Or sebagai pemain terbaik di dunia. Pada tahun 2021, Messi pindah ke klub Paris Saint-Germain (PSG) setelah meninggalkan Barcelona. Ia juga merupakan kapten tim nasional Argentina dan telah membantu timnas Argentina memenangkan Copa America pada tahun 2021.

Sumber: Tidak ada sumber yang relevan dengan pertanyaan ini.

In [67]:
print(chat_store.get_session_history(USER_ID, CONVERSATION_ID))

AI: SWDKLLJ untuk sepeda kumbang dan scooter di atas 50 cc sampai 250 cc adalah Rp 32.000,00 (tiga puluh dua ribu rupiah).

Sumber: [36/PMK.010/2008](https://www.ojk.go.id/id/regulasi/Documents/Pages/PMK-Nomor-36PMK.010-Tahun-2008-tentang-Besar-Santunan-dan-Sumbangan-Wajib-Dana-Kecelakaan-Lalu-Lintas-Jalan/menas13_1389258036.pdf#page=3)
Human: Apa itu uang pesangon?
AI: Uang pesangon adalah penghasilan yang dibayarkan oleh pemberi kerja kepada karyawan dengan nama dan dalam bentuk apapun sehubungan dengan berakhirnya masa kerja atau terjadi pemutusan hubungan kerja. Uang pesangon juga mencakup uang penghargaan masa kerja dan uang ganti kerugian.

Sumber: [112/KMK.03/2001](https://www.ojk.go.id/id/regulasi/Documents/Pages/KMK-Nomor-112.KMK.03.2001/112.KMK.03.2001.pdf#page=2)
Human: Apa itu LJKNB?
AI: LJKNB adalah singkatan dari Lembaga Jasa Keuangan Nonbank. LJKNB merujuk pada lembaga atau institusi keuangan yang tidak termasuk dalam kategori bank, namun tetap berperan dalam sektor jasa

**Clear Chat History**

In [59]:
chat_store.clear_all()

## **Evaluation**